# Diagnostyka raka skóry z wykorzystaniem analizy obrazów dermatologicznych - przygotowanie i analiza danych

Celem tego notatnika jest zapoznanie się z danym zbioru ISIC2019 oraz przygotowanie ich do dalszej pracy, czyli do trenowania i porównywania modeli DenseNet-121, DenseNet + SVM i DenseNet + RF. W tym notatniku przeanalizujemy metadane, przygotujemy obrazy do treningu oraz przetestujemy elementy potoku procesu treningu (podział z użyciem `StratifiedGroupKFold`, klasy `Dataset` i `DataLoader`). Końcowym efektem będzą odpowiednio przygotowane i podzielone dane do treningu.

## Konfiguracja środowiska

Importujemy potrzebne biblioteki.



In [ ]:
import os
import shutil
import glob
from tqdm import tqdm
from google.colab import drive, files
import kagglehub
import joblib
import json
from PIL import Image
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import multiprocessing

Definiujemy zmienne globalne.

In [ ]:
# parametry treningu
SEED = 42
BATCH_SIZE = 64                                   # wielkość partii danych
NUM_WORKERS = min(4, multiprocessing.cpu_count()) # liczba wątków procesora

# parametry transformacji obrazów
IMG_SIZE = 300                      # wymiar wejściowy obrazu
NORM_MEAN = [0.485, 0.456, 0.406]   # wartości średniej do normalizacji
NORM_STD = [0.229, 0.224, 0.225]    # wartości odchyleń do normalizacji

# parametry czyszczenia danych
VIGNETTE_THRESHOLD = 20      # próg czerni
EXTRA_CROP = 0.05            # margines dodatkowego przycięcia (5%)
MIN_FILL_RATIO = 0.95        # bezpiecznik dla zdjęć pełnokadrowych

# podział danych
TEST_SPLIT = 10              # 1/10 danych na zbiór testowy
CV_FOLDS = 5                 # liczba foldów walidacyjnych


Ustawiamy ziarno losości dla powtarzalności wyników.

In [ ]:
def set_seed(seed=42):
    """Ustawia ziarno losowości dla reprodukowalności wyników."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed(SEED)

Dodatkowo konfigurujemy wygląd wykresów w całym notatniku.

In [ ]:
# bazowy styl seaborn
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.1)

# konfiguracja parametrów matplotlib
plt.rcParams.update({
    # wymiary i jakość
    'figure.figsize': (10, 6),       # domyślny rozmiar
    'figure.dpi': 120,               # podgląd na ekranie
    'savefig.dpi': 300,              # jakość do druku
    'savefig.bbox': 'tight',         # automatyczne przycinanie marginesów przy zapisie
    'savefig.pad_inches': 0.1,       # mały bufor wokół przyciętego obrazka

    # tekst i czcionki
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans', 'Liberation Sans'],
    'axes.labelsize': 15,            # rozmiar nazw osi
    'axes.titlesize': 20,            # rozmiar tytułu wykresu
    'xtick.labelsize': 11,           # rozmiar liczb na osi X
    'ytick.labelsize': 11,           # rozmiar liczb na osi Y
    'legend.fontsize': 11,           # rozmiar tekstu w legendzie

    # linie i siatka
    'lines.linewidth': 2.5,          # grubsze linie
    'lines.markersize': 8,           # nieco większe kropki/punkty
    'grid.color': '#f0f0f0',         # bardzo jasna szarość siatki
    'grid.linewidth': 1.0,
    'axes.axisbelow': True,          # siatka pod spodem, dane na wierzchu!

    # estetyka ramki
    'axes.spines.top': False,        # usuń górną ramkę
    'axes.spines.right': False,      # usuń prawą ramkę
    'axes.linewidth': 1.2,           # grubość osi X i Y
    'legend.frameon': True,          # włącz ramkę legendy
    'legend.framealpha': 0.95,       # białe tło legendy
    'legend.facecolor': 'white'
})

# wyostrzenie w Colab
%config InlineBackend.figure_format = 'retina'

Konfigurujemy wszelki potrzebne ścieżki.

In [ ]:
# montowanie Google Drive
drive.mount('/content/drive')

# podajemy nazwę głównego folderu projektu
PROJECT_FOLDER_NAME = 'Implementacja'
BASE_DIR = os.path.join('/content/drive/MyDrive/Diagnostyka raka skóry z wykorzystaniem analizy obrazów dermatologicznych', PROJECT_FOLDER_NAME)

# podajemy ścieżkę w Colabie do zapisania pobieranych danych
DATA_ROOT_DIR = '/content/ISIC2019'
IMG_DIR = os.path.join(DATA_ROOT_DIR, 'images')
PROCESSED_DIR = os.path.join(DATA_ROOT_DIR, 'images_processed')

# definijemy podfoldery
MODELS_DIR = os.path.join(BASE_DIR, 'Modele')
RESULTS_DIR = os.path.join(BASE_DIR, 'Wyniki')
DATA_DIR = os.path.join(BASE_DIR, 'Dane')

# tworzymy foldery na Google Drive (jeśli nie istnieją)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Katalog roboczy projektu: {BASE_DIR}")
print(f"Dane będą zapisywane w: {DATA_ROOT_DIR}")

### Konfiguracja Kaggle API

Kaggle API jest nam potrzebne do pobrania danych bezpośrednio do notatnika. Folder ZIP jest bardzo obszernym plikiem i ręczne wgrywanie go na Google Drive może zająć bardzo dużo czasu.

Kofigurujemy ścieżkę dla Kaggle API. Klucz będzie zapisywany bezpośrednio w folderze projektu.

In [ ]:
kaggle_key_drive_path = os.path.join(BASE_DIR, 'kaggle.json')

Tworzymy w Colabie folder na plik z Kaggle API.

In [ ]:
kaggle_sys_dir = '/root/.kaggle'
os.makedirs(kaggle_sys_dir, exist_ok=True)

Wprowadzamy Kaggle API. Klucz ten należy utworzyć indywidualnie dla każdego użytkownika notatników z implementacją pracy inżynierskie. Aby go zdobyć należy:
1. Zalogować się na swój profil na stronie Kaggle.
2. Wejście w ustawienia swojego profilu (kilkamy na swoje zdjęcie profilowe i wybieramy ,,Settings'').
3. W sekcji ,,API'' klikamy ,,Generate New Token'', następnie wprowadzamy nazwę tokenu i klikamy ,,Generate''.
4. Kopiujemy kod z okienka ,,API TOKEN''.
5. W dowolnym edytorze tekstu (np. Notatniku) wprowadzamy tekst o następującej strykturze
$$\{\text{"username":"nazwa_uzytkownika_kaggle","key":"skopiowane_kaggle_api"}\}$$
i zapisujemy plik jako ,,kaggle.json''.
W poniższej funkcji zostaniemy poproszeni o wgranie utworzonego pliku.


In [ ]:
# sprawdzamy czy klucz jest już na Drive
if not os.path.exists(kaggle_key_drive_path):
    print(f"UWAGA: Nie znaleziono pliku kaggle.json w lokalizacji: {kaggle_key_drive_path}")
    print("Proszę przesłać plik kaggle.json teraz (zostanie zapisany na Drive):")

    # przesyłamy plik kaggle.json
    uploaded = files.upload()

    for filename in uploaded.keys():
        if filename == 'kaggle.json':
            # zapisujemy trwale na Google Drive
            with open(kaggle_key_drive_path, 'wb') as f:
                f.write(uploaded[filename])
            print(f"Zapisano kaggle.json w: {kaggle_key_drive_path}")
        else:
            print(f"Pominięto plik: {filename} (oczekiwano kaggle.json)")

## Pobranie danych

Pobieramy, rozpakowujemy i porządkujemy zbiór ISIC2019. Nie pobieramy obrazów na dysk Google Drive, lecz do pamięci Google Colab. Pozwoli to na sprawniejsze wczytywanie plików w notatniku.

In [ ]:
if os.path.exists(kaggle_key_drive_path):
    # instalacja klucza w systemie
    shutil.copy(kaggle_key_drive_path, os.path.join(kaggle_sys_dir, 'kaggle.json'))
    os.chmod(os.path.join(kaggle_sys_dir, 'kaggle.json'), 600)
    print("Klucz Kaggle API skonfigurowany.")

    # POBIERANIE I PORZĄDKOWANIE DANYCH

    # sprawdzamy, czy folder ze zdjęciami już istnieje i jest pełny
    # (np. czy nie uruchamiamy komórki drugi raz w tej samej sesji)
    if not os.path.exists(IMG_DIR):
        print("\nPobieranie danych z Kaggle...")
        # pobieramy do folderu tymczasowego
        path = kagglehub.dataset_download("andrewmvd/isic-2019")
        print(f"Pobrano dane do folderu tymczasowego: {path}")

        # tworzymy docelowe foldery na obrazy
        os.makedirs(IMG_DIR, exist_ok=True)
        os.makedirs(PROCESSED_DIR, exist_ok=True)

        # przenosimy metadane w folderu z danymi w Colabie
        for csv_file in glob.glob(os.path.join(path, "*.csv")):
            shutil.copy(csv_file, DATA_ROOT_DIR)
            print(f"Przeniesiono: {os.path.basename(csv_file)}")

        # szukamy rekursywnie zdjęć we wszystkich podfolderach pobranych danych
        jpg_files = glob.glob(os.path.join(path, "**", "*.jpg"), recursive=True)

        print(f"Znaleziono {len(jpg_files)} zdjęć.")

        for file_path in tqdm(jpg_files, desc="Kopiowanie zdjęć", unit="plik"):
            # przenosimy plik do wspólnego folderu
            shutil.copy(file_path, os.path.join(IMG_DIR, os.path.basename(file_path)))
        print("Przeniesiono zdjęcia do folderu z danymi.")

        # weryfikujemy liczbę plików
        num_images = len(os.listdir(IMG_DIR))
        print(f"\nSUKCES: Dane gotowe w {DATA_ROOT_DIR}")
        print(f"Liczba zdjęć w {IMG_DIR}: {num_images}")

    else:
        print("Dane są już pobrane i uporządkowane.")
        print(f"Lokalizacja zdjęć: {IMG_DIR}")

else:
    print("BŁĄD: Brak pliku kaggle.json. Nie można pobrać danych.")

## Analiza metadanych

Wczytujemy metadane i sprawdzamy informacje o nich.

In [ ]:
# ścieżki do metadanych
meta_path = os.path.join(DATA_ROOT_DIR, 'ISIC_2019_Training_Metadata.csv')
gt_path = os.path.join(DATA_ROOT_DIR, 'ISIC_2019_Training_GroundTruth.csv')

df_meta = pd.read_csv(meta_path)
df_gt = pd.read_csv(gt_path)

In [ ]:
df_meta.head()

In [ ]:
df_gt.head()

In [ ]:
df_meta.info()

In [ ]:
df_gt.info()

Zauważmy, że obie tabele mają wspólną kolumnę, zatem połączmy je w jeden plik z metadanymi.

In [ ]:
df = pd.merge(df_meta, df_gt, on='image', how='inner')
df.head()

In [ ]:
df.info()

Zauważmy, że mamy 25331 wierszy (tym samym tyle obrazów). Mamy jedyną kolumnę liczbową `age_approx`. Możemy zauważyć, że poza kolumną `image` mam do czynienia z brakami.

### Czyszczenie metadanych

Zajmijmy się najpierw uzupełnieniem braków. Zacznijmy od kolumny `lesion_id`. Zobaczmy na unikalne wartości tej kolumny.

In [ ]:
df['lesion_id'].nunique()

In [ ]:
df['lesion_id'].unique()

Zauważmy, że mamy 11847 unikalnych wartości w tej kolumnie. Oznacza to, że są zdjęcia, które przedstawiają tę samą zmianę lub są to zmiany jednego pacjenta. Załóżmy zatem, że braki obecne w tej kolumnie są unikalnymi zmianami. Pozwoli nam to zachować te zdjęcia w zbiorze. Braki w uzupełnimi odpowiednią wartością z kolumny `image`.

In [ ]:
df['lesion_id'] = df['lesion_id'].fillna(df['image'])
df.head()

W kolumnie `anatom_site_general` zawarte są informacje o położeniu zmiany. Możemy zatem uzupełnić braki wartością `unknown`, gdyż zakładamy, że przy tych zmianach nie podano informacji o położeniu.

In [ ]:
df['anatom_site_general'] = df['anatom_site_general'].fillna('unknown')
df.head()

Podobnie postępujemy z kolumną `sex`.

In [ ]:
df['sex'] = df['sex'].fillna('unknown')
df.head()

Kolumnę `age_approx`, która przedstawa przybliżony wiek pacentów, uzupełniamy medianą, gdyż jest to miara odporna na wartości odstające, które mogą być obecne wśród wartości kolumny.

In [ ]:
median_age = df['age_approx'].median()
df['age_approx'] = df['age_approx'].fillna(median_age)
df.head()

Sprawdźmy, jak wygląda zbiór po uzupełnieniu braków.

In [ ]:
df.info()

Przejdźmy teraz do scalenia kolumn ze zmianami skórnymi w jedną kolumnę dla łatwiejszego zarządzania klasami.

In [ ]:
lesion_cols = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
df['typ_zmiany'] = df[lesion_cols].idxmax(axis=1)
df.head()

Usuwamy zbędne kolumny.

In [ ]:
df = df.drop(columns=lesion_cols)
df.head()

Zmienimy teraz nazwy kolumn na polskie odpowiedniki.

In [ ]:
columns_pl_dict = {
    'image': 'id_zdjecia',
    'age_approx': 'wiek',
    'anatom_site_general': 'miejsce_zmiany',
    'lesion_id': 'id_zmiany',
    'sex': 'plec'
}

df = df.rename(columns=columns_pl_dict)
df.head()

Sprawdźmy teraz poszczególne wartości kolumn `miejsce_zmiany` i `plec`, w celu utworzenia słowników wprowadzających polskie odpowiedniki.

In [ ]:
df['miejsce_zmiany'].unique()

In [ ]:
df['plec'].unique()

Tworzymy słowniki i tłumaczymy wartości.

In [ ]:
miejsce_zmiany_dict = {
    'anterior torso': 'Przód tułowia',
    'upper extremity': 'Kończyna górna',
    'lower extremity': 'Kończyna dolna',
    'posterior torso': 'Plecy',
    'lateral torso': 'Bok tułowia',
    'head/neck': 'Głowa/Szyja',
    'palms/soles': 'Dłonie/Stopy',
    'oral/genital': 'Usta/Narządy płciowe',
    'unknown': 'Nieznane'
}

plec_dict = {
    'male': 'Mężczyzna',
    'female': 'Kobieta',
    'unknown': 'Nie podano'
}

df['miejsce_zmiany'] = df['miejsce_zmiany'].map(miejsce_zmiany_dict).astype('category')
df['plec'] = df['plec'].map(plec_dict).astype('category')

Sprawdźmy wyniki.

In [ ]:
df['miejsce_zmiany'].unique()

In [ ]:
df['plec'].unique()

Podobnie tłumaczymy nazwy wykrytych zmian skórnych.

In [ ]:
typ_zmiany_dict = {
    'MEL': 'Czerniak',                 # Melanoma
    'NV': 'Znamię barwnikowe',         # Melanocytic nevus
    'BCC': 'Rak podstawnokomórkowy',   # Basal cell carcinoma
    'AK': 'Rogowacenie słoneczne',     # Actinic keratosis
    'BKL': 'Łagodna zmiana',           # Benign keratosis lesion
    'DF': 'Włókniak',                  # Dermatofibroma
    'VASC': 'Zmiana naczyniowa',       # Vascular lesion
    'SCC': 'Rak kolczystokomórkowy',   # Squamous cell carcinoma
    'UNK': 'Nieznana zmiana'           # Unknown
}

df['typ_zmiany'] = df['typ_zmiany'].map(typ_zmiany_dict).astype('category')
df['typ_zmiany'].unique()

Zakodujmy teraz zmienną docelową `typ_zmiany` dla późniejszego procesu treningu.

In [ ]:
le = LabelEncoder()
df['target'] = le.fit_transform(df['typ_zmiany'])
df.head()

Sprawdźmy jeszcze, jak wygląda kodowanie.

In [ ]:
for i, label in enumerate(le.classes_):
    print(f"Klasa {i} --> {label}")

Zapisujemy obiekt `LabelEncoder` oraz kodowanie do późniejszej pracy.

In [ ]:
# ścieżki
encoder_path = os.path.join(DATA_DIR, 'label_encoder.joblib')
mapping_path = os.path.join(DATA_DIR, 'mapowanie_klas.json')

# zapisanie encodera
joblib.dump(le, encoder_path)

# wyciągnięcie mapowania i zapisanie JSON
class_map_dict = {int(index): label for index, label in enumerate(le.classes_)}

with open(mapping_path, 'w', encoding='utf-8') as f:
    json.dump(class_map_dict, f, ensure_ascii=False, indent=4)

print(f"Zapisano encoder do: {encoder_path}")
print(f"Zapisano słownik do: {mapping_path}")
print("Podgląd mapowania:")
print(class_map_dict)

Dodamy teraz kolumnę z nazwą pliku odpowiadającego danej zmianie. Ułatwi to późniejszą pracę.

In [ ]:
df['nazwa_pliku'] = df['id_zdjecia'].apply(lambda x: x if x.endswith('.jpg') else f"{x}.jpg")
df.head()

Sprawdźmy, jak teraz wygląda sytuacja z tabelą.

In [ ]:
df.info()

### Rozkład klas w zbiorze

Zobaczmy teraz, czy klasy w zbiorze są równomiernie rozłożone. Jeśli klasy nie są rozłożene równomiernie, to trzebw wtedy zastosować podział startyfikacyjny, czyli podział, który odwozorwje rozkład klas.

In [ ]:
counts = df['typ_zmiany'].value_counts()
ax = sns.barplot(x=counts.index,
                 y=counts.values,
                 palette="viridis",
                 hue=counts.index,
                 legend=False,
                 order=counts.index)

plt.title("Liczebność klas w zbiorze danych")
plt.ylabel("Liczba zdjęć")
plt.xlabel("Typ zmiany skórnej")
plt.xticks(rotation=45, ha='right')

plt.tight_layout()

# zapisanie wykresu
save_path = os.path.join(RESULTS_DIR, 'rozklad_klas.pdf')
plt.savefig(save_path)

# wyświetlenie wykresu
plt.show()

Klasy nie są rozłożone równomiernie, co musimy wziąć pod uwagę podczas podziału.

### Przygotowanie obrazów

Przejdźmy do analizy obrazów. Sprawdźmy teraz, jakie są wymiary obrazów oraz jakie są systemy kodowania kolorów w zbiorze.

In [ ]:
widths = []
heights = []
modes = [] # zbieranie schematów kodowania kolorów
missing_files = 0

for filename in tqdm(df['nazwa_pliku'], desc="Skanowanie zdjęć"):
    img_path = os.path.join(IMG_DIR, filename)

    try:
        with Image.open(img_path) as img:
            w, h = img.size
            widths.append(w)
            heights.append(h)
            modes.append(img.mode)

    except FileNotFoundError:
        missing_files += 1
    except Exception as e:
        print(f"Błąd przy pliku {filename}: {e}")

# wyznaczanie wartości unikalnych
unique_sizes = set(zip(widths, heights))
unique_modes = set(modes)

print("\nWyniki analizy:")
print(f"Przeanalizowano plików: {len(widths)}")
if missing_files > 0:
    print(f"Brakujących plików: {missing_files} (Sprawdzić ścieżki!)")

print(f"1. Unikalne rozmiary: {unique_sizes}")
print(f"2. Unikalne tryby kolorów: {unique_modes}")

# interpretacja
if 'RGB' in unique_modes and len(unique_modes) == 1:
    print("Wszystkie zdjęcia są w standardzie RGB (3 kanały).")
else:
    print("UWAGA: Wykryto niestandardowe tryby kolorów!")
    print(f"   Rozkład trybów: {pd.Series(modes).value_counts().to_dict()}")

Okazuje się, że wszystkie obrazy są w tym samym systemie kolorów, ale za to mamy różne wymiary zdjęć.

Zobaczmy przykłady obrazów z naszego zbioru.

In [ ]:
n_samples = 3
classes = df['typ_zmiany'].unique()

# tworzymy siatkę wykresów (wiersze = liczba chorób, kolumny = 3 przykłady)
fig, axes = plt.subplots(len(classes), n_samples, figsize=(8, 20))

for i, lesion in enumerate(classes):
    # pobieramy losowe 3 wiersze dla danej choroby
    probki = df[df['typ_zmiany'] == lesion].sample(n_samples)

    for j, (_, wiersz) in enumerate(probki.iterrows()):
        # wczytujemy zdjęcie bezpośrednio
        sciezka = os.path.join(IMG_DIR, wiersz['nazwa_pliku'])
        img = Image.open(sciezka)
        img = img.resize((int(IMG_SIZE * 0.75), int(IMG_SIZE * 0.75)))

        # rysujemy
        ax = axes[i, j]
        ax.imshow(img)
        ax.axis('off')

        # tytuł tylko po lewej stronie
        if j == 0:
            ax.set_title(lesion, loc='left', fontsize=15)

plt.tight_layout()

# zapisywanie
save_path = os.path.join(RESULTS_DIR, 'przyklady_zmian.pdf')
plt.savefig(save_path)

plt.show()

Zauważmy, że wybrane przykłady zawierają czarną wynietę lub czarne rogi. Spróbujemy się ich częściowo pozbyć, aby trenowane modele lepiej uczyły się zmian skórnych.

Tworzymy funkcję do wycinania winiety.

In [ ]:
def crop_vignette(image_path, output_path, threshold=20, extra_crop_percent=0.05, min_fill_ratio=0.95):
    """
    Funkcja usuwająca czarne ramki (winiety) z obrazu z dodatkowym marginesem bezpieczeństwa
    oraz mechanizmem chroniącym zdjęcia pełnokadrowe.

    Parametry:
    - image_path: ścieżka do pliku wejściowego.
    - output_path: ścieżka zapisu pliku wynikowego.
    - threshold: próg jasności (0-255). Piksele ciemniejsze niż ta wartość są traktowane jako tło.
    - extra_crop_percent: ułamek (np. 0.05 = 5%), o jaki dodatkowo zmniejszamy ramkę z każdej strony,
      aby usunąć postrzępione krawędzie lub pozostałości winiety w rogach.
    - min_fill_ratio: współczynnik wypełnienia (zakres 0.0 - 1.0). Określa próg decyzyjny,
      czy zdjęcie w ogóle wymaga przycinania. Jeśli wykryty obszar treści zajmuje większą część
      zdjęcia niż ta wartość (np. > 95%), algorytm uznaje, że winieta nie występuje (lub jest pomijalna)
      i rezygnuje z przycinania, aby nie uszkodzić zmian skórnych dotykających krawędzi.
    """

    # wczytujemy obraz z dysku
    img = cv2.imread(image_path)

    # jeśli plik jest uszkodzony lub nie istnieje, przerywamy
    if img is None:
        return False, None

    # przekształcamy obraz do skali szarości i wyznaczmy obszar zdjęcia
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h_img, w_img = gray.shape
    total_area = h_img * w_img

    # tworzymy maskę binarną:
    # - piksele > threshold (treść) dostają wartość 255 (biały)
    # - piksele <= threshold (tło/winieta) dostają wartość 0 (czarny)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # znajdujemy położenie treści
    # cv2.findNonZero zwraca listę współrzędnych wszystkich białych pikseli na masce
    coords = cv2.findNonZero(mask)

    # jeśli nie znaleziono żadnych jasnych punktów (zdjęcie jest całe czarne), kończymy
    if coords is None:
        return False, None

    # znajdujemy najmniejszy prostokąt (x, y, szerokość, wysokość), który obejmuje wszystkie jasne punkty
    x, y, w, h = cv2.boundingRect(coords)

    # obliczamy obszar czystego obrazu oraz współczynnik wypełnienia
    rect_area = w * h
    fill_ratio = rect_area / total_area

    # jeśli treść zajmuje prawie cały obraz (> 95%), to znaczy, że nie ma winiety,
    # albo są tylko mikro-paski, których nie warto ruszać, by nie uciąć zmiany
    if fill_ratio > min_fill_ratio:
        return False, None

    # ETAP PRZYCINANIA

    # obliczamy ile pikseli uciąć dodatkowo z każdej strony (5% szerokości/wysokości znalezionej ramki)
    margin_w = int(w * extra_crop_percent)
    margin_h = int(h * extra_crop_percent)

    # wyznaczamy nowe współrzędne ramki, zawężając ją do środka:
    # przesuwamy początek X w prawo i Y w dół
    new_x = x + margin_w
    new_y = y + margin_h

    # zmniejszamy szerokość i wysokość o marginesy z obu stron (lewo+prawo, góra+dół)
    new_w = w - (2 * margin_w)
    new_h = h - (2 * margin_h)

    # jeśli zdjęcie było bardzo małe lub margines był za duży i "zjadł" cały obraz (wymiar <= 0),
    # to cofamy się do wersji podstawowej (bez cięcia).
    if new_w <= 0 or new_h <= 0:
        new_x, new_y, new_w, new_h = x, y, w, h

    # sprawdzamy czy przycięcie jest konieczne
    # pobieramy oryginalne wymiary obrazu
    h_img, w_img = gray.shape

    # czy nowa, wyliczona ramka jest mniejsza niż oryginalny obraz?
    # jeśli tak, wykonujemy fizyczne cięcie obrazu
    if (new_w < w_img) or (new_h < h_img):
        # wycinanie fragmentu tablicy
        cropped_img = img[new_y:new_y+new_h, new_x:new_x+new_w]

        # zapisujemy wynik na dysk
        cv2.imwrite(output_path, cropped_img)

        # zwracamy sukces oraz informację, ile pikseli ucięto z każdej strony (góra, dół, lewo, prawo)
        return True, (new_y, h_img-(new_y+new_h), new_x, w_img-(new_x+new_w))

    # jeśli nie trzeba było ciąć (ramka pokrywa się z całym obrazem), zwracamy False
    return False, None

Dokonujemy przycięcia.

In [ ]:
# filtrujemy pliki, biorąc tylko te z rozszerzeniem .jpg lub .png
image_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(('.jpg', '.png'))]

cropped_count = 0
copied_count = 0

print(f"Start: {len(image_files)} zdjęć. Dodatkowe cięcie: {EXTRA_CROP*100}%")

# pętla po wszystkich zdjęciach z użyciem paska postępu
for f in tqdm(image_files, desc="Przetwarzanie"):

    # tworzenie pełnych ścieżek
    src_path = os.path.join(IMG_DIR, f)
    dst_path = os.path.join(PROCESSED_DIR, f)

    # wywołujemy cięcie
    is_cropped, _ = crop_vignette(src_path, dst_path, threshold=VIGNETTE_THRESHOLD, extra_crop_percent=EXTRA_CROP, min_fill_ratio=MIN_FILL_RATIO)

    if is_cropped:
        # jeśli funkcja wykonała cięcie i zapisała plik -> zwiększ licznik
        cropped_count += 1
    else:
        # jeśli funkcja nie przycięła zdjęcia (zwróciła False) -> kopiujemy oryginał bez zmian
        shutil.copy(src_path, dst_path)
        copied_count += 1

print("\nZakończono przycinanie.")
print(f"Przycięto: {cropped_count}")
print(f"Skopiowano (bez zmian): {copied_count}")

Wyświetlmy losowe 5 przykładów, aby zobaczyć efekty przycięcia.

In [ ]:
if not image_files:
    print("Błąd: Folder wyjściowy jest pusty! Sprawdź, czy skrypt przetwarzający zadziałał poprawnie.")
else:
    candidates = []

    for f in image_files:
        p_orig = os.path.join(IMG_DIR, f)
        p_proc = os.path.join(PROCESSED_DIR, f)

        # jeśli rozmiar pliku jest inny od oryginalnego, to zapisujemy zdjęcie
        if os.path.getsize(p_orig) != os.path.getsize(p_proc):
            candidates.append(f)

    # losujemy 5 zdjęć do wyświetlenia
    samples_pool = candidates if len(candidates) >= 3 else image_files
    selected_files = random.sample(samples_pool, min(3, len(samples_pool)))

    # WYŚWIETLENIE ZDJĘĆ
    # tworzymy siatkę obrazów
    fig, axes = plt.subplots(len(selected_files), 2, figsize=(8, 4 * len(selected_files)))

    # jeśli mamy tylko 1 zdjęcie, `axes` nie jest tablicą,
    # więc zamykamy go w listę, aby pętla for poniżej zadziałała bez błędu.
    if len(selected_files) == 1:
        axes = [axes]

    # pętla rysująca
    for i, filename in enumerate(selected_files):
        path_orig = os.path.join(IMG_DIR, filename)
        path_proc = os.path.join(PROCESSED_DIR, filename)

        # wczytujemy obrazy
        img_orig = Image.open(path_orig)
        img_proc = Image.open(path_proc)

        # tworzymy kolumnę z oryginalnymi zdjęciami
        axes[i, 0].imshow(img_orig)
        # wyświetlamy wymiary oryginału w tytule
        axes[i, 0].set_title(f"Oryginał\n{img_orig.size}", fontsize=10, color='gray')
        axes[i, 0].axis('off') # ukrywamy osie

        # tworzymy kolumnę z przyciętymi obrazami
        # obliczamy, o ile procent zmniejszyła się powierzchnia zdjęcia.
        area_orig = img_orig.width * img_orig.height
        area_proc = img_proc.width * img_proc.height
        reduction = (1 - (area_proc / area_orig)) * 100

        axes[i, 1].imshow(img_proc)

        # formatowanie tytułu:
        # jeśli redukcja > 0%, to znaczy, że winieta została wycięta -> Kolor ZIELONY + info o %
        # jeśli 0%, to zdjęcie było tylko skopiowane -> Kolor CZARNY + "BEZ ZMIAN"
        if reduction > 0:
            title_text = f"PRZYCIĘTO (Redukcja: {reduction:.1f}%)\n{img_proc.size}"
            title_col = 'green'
        else:
            title_text = f"BEZ ZMIAN\n{img_proc.size}"
            title_col = 'black'

        axes[i, 1].set_title(title_text, fontsize=10, fontweight='bold', color=title_col)
        axes[i, 1].axis('off')

    plt.tight_layout()

    # zapisywanie
    save_path = os.path.join(RESULTS_DIR, 'przyklady_zmian_przyciete.pdf')
    plt.savefig(save_path)

    plt.show()

Obraz wyglądają lepiej po przycięciu. Obrazy są zatem przygotowane do dalszej pracy.

## Test klasy `Dataset` i augumentacji danych

Definujemy klasę potomną klasy `Dataset`, która będzie wczytywać poszczególne obrazy zbioru ISIC2019.

In [ ]:
class ISICDataset(Dataset):
    """
    Niestandardowy Dataset PyTorch do wczytywania obrazów dermatologicznych ISIC.

    Klasa wczytuje obrazy na podstawie ścieżek z DataFrame, konwertuje je do RGB
    i zwraca w formie gotowej do przetworzenia przez model.
    """

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Inicjalizuje dataset.

        Args:
            dataframe (pd.DataFrame): Ramka danych zawierająca co najmniej kolumny:
                                      - 'nazwa_pliku': nazwa pliku z rozszerzeniem (np. 'IMG_1.jpg')
                                      - 'target': numeryczna etykieta klasy (int).
            root_dir (str): Ścieżka do katalogu głównego, w którym znajdują się obrazy.
            transform (callable, optional): Opcjonalne transformacje (np. augmentacja, normalizacja)
                                            aplikowane na obrazie. Domyślnie None.
        """
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        """Zwraca całkowitą liczbę próbek w zbiorze."""
        return len(self.df)

    def __getitem__(self, idx):
        """
        Pobiera pojedynczą próbkę danych o podanym indeksie.

        Args:
            idx (int/tensor): Indeks próbki do pobrania.

        Returns:
            tuple: Krotka (image, label), gdzie:
                   - image (PIL.Image lub Tensor): Obraz po transformacjach.
                   - label (int): Numeryczna etykieta klasy.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # pobranie nazwy pliku
        img_name = self.df.iloc[idx]['nazwa_pliku']
        img_path = os.path.join(self.root_dir, img_name)

        # wczytanie obrazu i konwersja na RGB (dla bezpieczeństwa)
        image = Image.open(img_path).convert('RGB')

        # pobranie etykiety numerycznej (target)
        label = self.df.iloc[idx]['target']

        if self.transform:
            image = self.transform(image)

        return image, label

Definiujemy również transformacje augumentacji na zbiorze treningowym i walidacyjnym.

In [ ]:
working_size = int(IMG_SIZE * 1.25)

train_transforms = transforms.Compose([
    # losujemy wycinek od 80% do 100% powierzchni oryginalnego obrazu
    transforms.RandomResizedCrop(working_size, scale=(0.8, 1.0), ratio=(0.9, 1.1)),

    # losowe odbicia lustrzane pionowo i poziomo
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),

    # delikatna zmiana koloru
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.0),

    # obrót zdjęcia o kąt z przedziału (-15 stopni, 15 stopni)
    transforms.RandomRotation(15),

    # wycinamy środek obrazu w docelowych wymiarach
    transforms.CenterCrop(IMG_SIZE),

    # przeniesienie na tensor i normalizacja
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

val_transforms = transforms.Compose([
    # zmiana wymiarów
    transforms.Resize(working_size),
    # wycinamy środek obrazu w docelowych wymiarach
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

Spróbujmy zwizualizować jedno zdjęcie w 10 różnych wariantach, aby sprawdzić działanie augumentacji.

Dodatkowo zbudujemy funkcję, która odwraca normalizację. Będziemy mogli w ten sposób zobaczyć zmianę w czytelniejszej formie.

In [ ]:
def denormalize(tensor, mean, std):
    """
    Cofnięcie procesu normalizacji na tensorze obrazu (przywrócenie oryginalnej skali kolorów).

    W uczeniu głębokim obrazy wejściowe są często standaryzowane według wzoru:
    $output = (input - mean) / std$. Funkcja ta odwraca tę operację ($output = input * std + mean$),
    co jest niezbędne do poprawnej wizualizacji obrazów, które zostały wcześniej
    przetworzone przez transformacje PyTorch (np. `transforms.Normalize`).

    Args:
        tensor (torch.Tensor): Tensor obrazu o kształcie (C, H, W).
        mean (tuple/list): Średnie użyte do normalizacji dla każdego kanału (R, G, B).
        std (tuple/list): Odchylenia standardowe użyte do normalizacji dla każdego kanału (R, G, B).

    Returns:
        torch.Tensor: Zdenormalizowany tensor gotowy do konwersji na format obrazu (np. PIL lub NumPy).
    """
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

In [ ]:
dataset = ISICDataset(df, PROCESSED_DIR, transform=train_transforms)

idx = 12345 # indeks zdjęcia do testu augumentacji
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle("Przykład augumentacji danych", fontsize=20)
axes = axes.flatten()

for i in range(10):
    img_tensor, _ = dataset[idx]
    img = denormalize(img_tensor, NORM_MEAN, NORM_STD).permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)

    axes[i].imshow(img)
    axes[i].axis('off')

# zapisywanie
save_path = os.path.join(RESULTS_DIR, 'test_augumentacji.pdf')
plt.savefig(save_path)

plt.show()

Wygląda na to, że transformacje działają poprawnie.

## Test klasy `StratifiedGroupKFold`

W procesie treningu będzie korzystać z mechanizmu walidacji krzyżowej z podziałem na pacjenta. Podział ten jest realizowany przez klasę `StratifiedGroupKFold`. Sprawdźmy poprawność tego podziału, czyli sprawdźmy, czy zmiany jednego pacjenta nie pojawiają się różnych podzbiorach. Dodatkowo sprawdzimy, czy klasa poprawnio odwzorowuje rozkład klas w poszczególnych foldach.

Konfigurujemy potrzebne dane.

In [ ]:
X = df['nazwa_pliku']
y = df['target']
groups = df['id_zmiany']

# inicjalizujemy istancję klasy
sgkf = StratifiedGroupKFold(n_splits=CV_FOLDS)

Sprawdzamy, czy dochodzi do przecieku danych. Dodatkowo zbierzemy informacje o rozkładzie klas.

In [ ]:
folds_data = [] # do przechowania indeksów
fold_stats = [] # do przechowania statystyk procentowych


for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):

    # tworzymy podzbiory
    train_subset = df.iloc[train_idx]
    val_subset = df.iloc[val_idx]

    # sprawdzamy, czy dochodzi do przecieku danych
    # czy istnieją zdjęcia przypisane do obu zbiorów?
    train_groups = set(train_subset['id_zmiany'])
    val_groups = set(val_subset['id_zmiany'])
    overlap = train_groups.intersection(val_groups)
    num_overlap = len(overlap)

    status = "OK" if num_overlap == 0 else "BŁĄD!"
    print(f"Fold {fold+1}: Trening={len(train_subset)}, Walidacja={len(val_subset)} | Przeciek: {num_overlap} pacjentów -> {status}")

    # zapisujemy indeksy
    folds_data.append((train_idx, val_idx))

    # liczymy procentowy udział klas w zbiorze walidacyjnym tego foldu
    counts = val_subset['typ_zmiany'].value_counts(normalize=True) * 100
    counts.name = f"Fold {fold+1}"
    fold_stats.append(counts)

Nie dochodzi do przecieku danych. Do informacji i rozkładzie dodajmy jeszcze rozkład klas w całym zbiorze.

In [ ]:
global_counts = df['typ_zmiany'].value_counts(normalize=True) * 100
global_counts.name = 'Cały zbiór'
fold_stats.append(global_counts)

Zbudujmy ramkę danych i posortujmy kolumny po częsitości występowania w całym zbiorze.

In [ ]:
dist_df = pd.DataFrame(fold_stats)

# sortujemy kolumny od najczęstszej do najrzadszej względem wiersza rozkładu całego zbioru
sorted_columns = dist_df.loc['Cały zbiór'].sort_values(ascending=False).index
dist_df = dist_df[sorted_columns]

dist_df.round(2)

Zwizualizujemy odwzorowanie klas.

In [ ]:
ax = dist_df.plot(kind='bar', stacked=True, colormap='viridis', width=0.8)

plt.title("Rozkład klas w foldach w porównaniu z całym zbiorem")
plt.xlabel("Zbiór danych")
plt.ylabel("Udział procentowy (%)")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title="Typ zmiany")
plt.xticks(rotation=45)
# plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()

# zapisywanie
save_path = os.path.join(RESULTS_DIR, 'rozklad_klas_foldy.pdf')
plt.savefig(save_path)

plt.show()

Możemy zatem stwierdzić, że klasa działa tak, jak tego oczekiwaliśmy.

## Test klasy `DataLoader`

Przejdźmy do przetestowania obiektu `DataLoader` (klasy wprowadzającej dane do modelu w procesie uczenia). Sprawdzimy, czy wszystko będzie wykonywane poprawnie na jednym z podzbiorów podziału.

Tworzymy zbiór danych z jednego folda podziału.

In [ ]:
# pobieramy indeksy dla pierwszego foldu z generatora
train_idx, val_idx = next(sgkf.split(X, y, groups))

# tworzymy podzbiory
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)

print(f"Podział danych:")
print(f" -> Trening: {len(train_df)} zdjęć")
print(f" -> Walidacja: {len(val_df)} zdjęć")

Inicjalizujemy instancje `ISICDataset` na przygotowanych zbiorach.

In [ ]:
train_dataset = ISICDataset(train_df, PROCESSED_DIR, transform=train_transforms)
val_dataset = ISICDataset(val_df, PROCESSED_DIR, transform=val_transforms)

Tworzymy instancje klasy `DataLoader` dla zbioru treningowego i testowego.

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,          # mieszamy dane treningowe
    num_workers=NUM_WORKERS,
    pin_memory=True,       # przyspiesza transfer do GPU
    drop_last=True         # odrzucamy ostatnią niepełną partię
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,         # w walidacji nie mieszamy
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"Liczba partii w epoce (trening): {len(train_loader)}")
print(f"Liczba partii w epoce (walidacja): {len(val_loader)}")

Zbieramy informacji o jednej partii.

In [ ]:
images, labels = next(iter(train_loader))

print(f"Analiza pobranej partii:")
print(f" -> Kształt obrazów: {images.shape}  (liczność partii, C, H, W)")
print(f" -> Kształt etykiet: {labels.shape}")
print(f" -> Przykładowe etykiety: {labels[:10].tolist()}")

Wygląda na to, że instancje `DataLoader` działają poprawnie. Zobaczmy jeszcze, jak wygląda partia.

In [ ]:
# tworzymy siatkę
grid_img = make_grid(images[:BATCH_SIZE], nrow=8, padding=2, normalize=False)

# przenosimy na cpu i zmieniamy kolejność wymiarów (C, H, W) -> (H, W, C)
grid_img = grid_img.permute(1, 2, 0).cpu().numpy()

# odwracamy normalizację
grid_img = grid_img * NORM_STD + NORM_MEAN

# sprowadzamy zakers do  [0,1]
grid_img = np.clip(grid_img, 0, 1)

# tworzymy wykres
plt.imshow(grid_img)
plt.title(f"Przykładowa partia danych treningowych")
plt.axis('off')

# zapisujemy
save_path = os.path.join(RESULTS_DIR, 'przyklad_partii.pdf')
plt.savefig(save_path)

plt.show()

Możemy zatem przejść do podzielenia danych do dalszych etapów projektu.

## Ostateczny podział danych do trenowania modeli

Przy podziale danych oddzielimy 10% zbioru na zbiór testowy, a pozostałe 90% podzielimy na zbiór treningowy i walidacyjny. Wszystko przy pomocy `StratifiedGroupKFold`, aby uzyskać podział na poziomie pacjenta/zmiany. Instrukcje podziały (numery foldów dla każdej obserwacji) zapiszemy w tabeli z metadanymi.

Tworzymy końcową ramkę metadanych i inicjalizujemy w niej kolumny przypisujące numer foldu danej obserwacji.

In [ ]:
df_final = df.reset_index(drop=True)

df_final['typ_puli'] = 'trening'  # domyślnie wszystko trafia do puli treningowej
df_final['nr_podzbioru'] = -1     # -1 oznacza zbiór testowy

df_final.head()

Dokonujemy wydzielenia zbioru testowego.

In [ ]:
test_splitter = StratifiedGroupKFold(n_splits=TEST_SPLIT)

# generujemy indeksy podziału
generator = test_splitter.split(df_final, y, groups)

# pobieramy indeksy pierwszego podziału
train_idx, test_idx = next(generator)

# przypisujemy wybranym ideksom kategorie zbioru testowego
df_final.loc[test_idx, 'typ_puli'] = 'test'

df_final.head()

Dzielimy resztę na zbiór treningowy i walidacyjny w poszczególnych foldach.

In [ ]:
# tworzymy tymczasowy widok tylko na dane treningowe
train_mask = df_final['typ_puli'] == 'trening'
df_train_temp = df_final[train_mask].reset_index(drop=True)

cv_splitter = StratifiedGroupKFold(n_splits=CV_FOLDS)

# dzielimy df_train_temp na foldy
for fold_num, (t_idx, v_idx) in enumerate(cv_splitter.split(df_train_temp, df_train_temp['target'], df_train_temp['id_zmiany'])):

    # pobieramy ID zmian, które trafiły do walidacji w tym foldzie
    val_lesion_ids = df_train_temp.loc[v_idx, 'id_zmiany'].values

    # przenosimy tę informację do głównej tabeli
    # przypisujemy numer foldu walidacyjnego tym wierszom, które są w puli treningowej
    # i są w zbiorze walidacyjnym powyższego odziału
    condition = (df_final['typ_puli'] == 'trening') & (df_final['id_zmiany'].isin(val_lesion_ids))
    df_final.loc[condition, 'nr_podzbioru'] = fold_num

df_final.head()

Sprawdźmy liczbę obserwacji w każdym z podzbiorów.

In [ ]:
df_final.groupby(['typ_puli', 'nr_podzbioru']).size()

Sprawdźmy teraz, czy po takim podziale został poprawnie odwzorowany rozkład klas.

In [ ]:
# obliczamy rozkład klas dla zbioru testowego
test_counts = df_final[df_final['typ_puli'] == 'test']['typ_zmiany'].value_counts(normalize=True) * 100

# obliczamy rozkład dla każdego foldu
fold_counts = {}
for i in range(CV_FOLDS):
    # wybieramy wiersze należące do konkretnego foldu
    fold_subset = df_final[df_final['nr_podzbioru'] == i]
    fold_counts[f'Fold {i}'] = fold_subset['typ_zmiany'].value_counts(normalize=True) * 100

# łączymy wszystko w jedną ramkę danych
comparison_df = pd.DataFrame({
    'Cały zbiór': global_counts,
    'Zbiór testowy': test_counts,
    **fold_counts # rozpakowujemy słownik z foldami
})

Zobaczmy rozkład klas w podziale.

In [ ]:
comparison_df = comparison_df.sort_values(by='Cały zbiór', ascending=False)

comparison_df.round(2)

Upewnijmy się, że nie ma zbyt dużych odchyleń w rozkładach klas.

In [ ]:
# szukamy maksymalnego odchylenia
# odejmujemy od każdej kolumny wartość dla całego zbioru i szukamy maksymalnej różnicy
diff_df = comparison_df.subtract(comparison_df['Cały zbiór'], axis=0).abs()
max_diff = diff_df.max().max()

print(f"Maksymalne odchylenie od średniej globalnej: {max_diff:.2f} punktu procentowego.")
if max_diff < 1.5:
    print("Podział jest bardzo dobrze zbalansowany.")
else:
    print("Uwaga: Istnieją pewne dysproporcje w podziale.")

Zapisujemy ostatecznie przetworzone metadane.

In [ ]:
save_path = os.path.join(DATA_DIR, 'ISIC_metadata_processed.csv')
df_final.to_csv(save_path, index=False)
print(f"Zapisano metadae do: {save_path}")

Dokonaliśmy zatem ostatecznego podziału danych do treningu. Zapisaliśmy wszelkie metadane potrzebne w kolejnych etapach projekty. Wszlekie elementy potoku pracy (podział danych, klasy `Dataset` i `DataLoader`) działają poprawnie.